In [0]:
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn as nn 
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset, TensorDataset

In [4]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/ECE 247/project/project

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/ECE 247/project/project


In [0]:
import numpy as np
X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")
person_train_valid = np.load("person_train_valid.npy")
X_train_valid = np.load("X_train_valid.npy")
y_train_valid = np.load("y_train_valid.npy")
person_test = np.load("person_test.npy")
y_train_valid = y_train_valid - 769
y_test = y_test - 769


In [6]:
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))


Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [0]:

def datatime(X,X_test,sub_idx=None, time_period=1000):
    # different period of time
    X_train_valid_time = X[:, :, :time_period]
    X_test_time = X_test[:, :, :time_period]
    return X_train_valid_time,X_test_time

In [8]:
use_cuda = torch.cuda.is_available()
device = torch.device('cuda:0' if use_cuda else 'cpu')
print(device)
torch.cuda.get_device_name(0)

cuda:0


'Tesla P100-PCIE-16GB'

In [9]:
dtype = torch.cuda.FloatTensor


X_train_valid_cur, X_test_cur= datatime(X_train_valid, X_test,time_period=600)

perm = np.random.permutation(X_train_valid_cur.shape[0])
numTrain = int(0.9*X_train_valid_cur.shape[0])
numVal = X_train_valid_cur.shape[0] - numTrain
Xtrain = X_train_valid_cur[perm[0:numTrain]]
ytrain = y_train_valid[perm[0:numTrain]]
Xval = X_train_valid_cur[perm[numTrain:]]
yval = y_train_valid[perm[numTrain:]]


X_train_reshape = np.expand_dims(Xtrain, axis=1)
X_val_reshape = np.expand_dims(Xval, axis=1)
print(X_train_reshape.shape)


x_train_tensor = torch.from_numpy(X_train_reshape).float()
y_train_tensor = torch.from_numpy(ytrain).float()

train_data = TensorDataset(x_train_tensor, y_train_tensor)

x_val_tensor = torch.from_numpy(X_val_reshape).float()
y_val_tensor = torch.from_numpy(yval).float()

val_data = TensorDataset(x_val_tensor, y_val_tensor)

print(train_data[0])
batchsize = 32
train_loader = DataLoader(dataset=train_data, batch_size=batchsize, shuffle=True)
val_loader = DataLoader(dataset=val_data, batch_size=batchsize, shuffle=True)

N,C,H,W = X_train_reshape.shape
x = Variable(torch.randn(N,C,H,W))
y= Variable(torch.Tensor(ytrain), requires_grad = False)
x.type(dtype)
y.type(dtype)




(1903, 1, 22, 600)
(tensor([[[15.9180, 15.4297,  7.5684,  ...,  0.6836,  8.0078,  9.2773],
         [ 7.0312,  6.4941,  2.3926,  ..., -0.1465,  8.2031,  9.0332],
         [13.2324, 11.1816,  4.2969,  ..., -2.2461,  6.0059,  8.7402],
         ...,
         [13.5254, 11.3770,  8.1055,  ..., -4.4922,  1.6113,  3.5645],
         [13.9160,  9.8145,  7.2266,  ..., -4.8340, -0.1465,  1.9043],
         [11.3770,  8.3496,  5.4199,  ..., -7.1777, -1.0742, -1.0254]]]), tensor(1.))


tensor([1., 1., 2.,  ..., 1., 0., 1.], device='cuda:0')

In [0]:
class InceptionNet(nn.Module):
    def __init__(self):
        super(InceptionNet, self).__init__()
        self.conv0 = nn.Conv2d(in_channels=22, out_channels=32, kernel_size=(1,9), stride=(1,1), padding=(0,4))
        self.conv0_bn = nn.BatchNorm2d(32)
        self.conv1 = nn.Conv2d(in_channels=22, out_channels=32, kernel_size=(1,7), stride=(1,1), padding=(0,3))
        self.conv1_bn = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(in_channels=22, out_channels=32, kernel_size=(1,5), stride=(1,1), padding=(0,2))
        self.conv2_bn = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(in_channels=22, out_channels=32, kernel_size=(1,3), stride=(1,1), padding=(0,1))
        self.conv3_bn = nn.BatchNorm2d(32)
        self.conv4 = nn.Conv2d(in_channels=22, out_channels=32, kernel_size=(1,1), stride=(1,1))
        self.conv4_bn = nn.BatchNorm2d(32)
        
        self.conv5 = nn.Conv2d(in_channels=160, out_channels=64, kernel_size=(1,9), stride=(1,1), padding=(0,4))
        self.conv5_bn = nn.BatchNorm2d(64)       
        self.conv6 = nn.Conv2d(in_channels=160, out_channels=64, kernel_size=(1,7), stride=(1,1), padding=(0,3))
        self.conv6_bn = nn.BatchNorm2d(64)
        self.conv7 = nn.Conv2d(in_channels=160, out_channels=64, kernel_size=(1,5), stride=(1,1), padding=(0,2))
        self.conv7_bn = nn.BatchNorm2d(64)
        self.conv8 = nn.Conv2d(in_channels=160, out_channels=64, kernel_size=(1,3), stride=(1,1), padding=(0,1))
        self.conv8_bn = nn.BatchNorm2d(64)        
        self.conv9 = nn.Conv2d(in_channels=160, out_channels=64, kernel_size=(1,1), stride=(1,1), padding=(0,0))
        self.conv9_bn = nn.BatchNorm2d(64)
        
        self.conv10 = nn.Conv2d(in_channels=320, out_channels=128, kernel_size=(1,9), stride=(1,1), padding=(0,4))
        self.conv10_bn = nn.BatchNorm2d(128)       
        self.conv11 = nn.Conv2d(in_channels=320, out_channels=128, kernel_size=(1,7), stride=(1,1), padding=(0,3))
        self.conv11_bn = nn.BatchNorm2d(128)
        self.conv12 = nn.Conv2d(in_channels=320, out_channels=128, kernel_size=(1,5), stride=(1,1), padding=(0,2))
        self.conv12_bn = nn.BatchNorm2d(128)
        self.conv13 = nn.Conv2d(in_channels=320, out_channels=128, kernel_size=(1,3), stride=(1,1), padding=(0,1))
        self.conv13_bn = nn.BatchNorm2d(128)        
        self.conv14 = nn.Conv2d(in_channels=320, out_channels=128, kernel_size=(1,1), stride=(1,1), padding=(0,0))
        self.conv14_bn = nn.BatchNorm2d(128)
        
        self.relu = nn.ELU(inplace=True) #nn.ReLU(inplace=True)
        
        self.fc = nn.Linear(in_features=640*15,  out_features=4)
        self.maxpool = nn.MaxPool2d(kernel_size=(1,2), stride=(1,2), padding=(0,0))
        self.avgpool = nn.AvgPool2d(kernel_size=(1,10), stride=(1,10), padding=(0,0))
        
        
        
    def forward(self, X):
        # Nx1x22x1000
        xi = X.permute(0,2,1,3)
        x = self.conv0(xi)
        x = self.conv0_bn(x)
        x0 = self.relu(x)
        x = self.conv1(xi)
        x = self.conv1_bn(x)
        x1 = self.relu(x)
        x = self.conv2(xi)
        x = self.conv2_bn(x)
        x2 = self.relu(x)
        x = self.conv3(xi)
        x = self.conv3_bn(x)
        x3 = self.relu(x)
        x = self.conv4(xi)
        x = self.conv4_bn(x)
        x4 = self.relu(x)
        x = torch.cat((x0,x1,x2,x3,x4),1)
        
        xi = self.maxpool(x)
                
        x = self.conv5(xi)
        x = self.conv5_bn(x)
        x0 = self.relu(x)
        x = self.conv6(xi)
        x = self.conv6_bn(x)
        x1 = self.relu(x)
        x = self.conv7(xi)
        x = self.conv7_bn(x)
        x2 = self.relu(x)
        x = self.conv8(xi)
        x = self.conv8_bn(x)
        x3 = self.relu(x)
        x = self.conv9(xi)
        x = self.conv9_bn(x)
        x4 = self.relu(x)
        x = torch.cat((x0,x1,x2,x3,x4),1)
       
        xi = self.maxpool(x)

        x = self.conv10(xi)
        x = self.conv10_bn(x)
        x0 = self.relu(x)
        x = self.conv11(xi)
        x = self.conv11_bn(x)
        x1 = self.relu(x)
        x = self.conv12(xi)
        x = self.conv12_bn(x)
        x2 = self.relu(x)
        x = self.conv13(xi)
        x = self.conv13_bn(x)
        x3 = self.relu(x)
        x = self.conv14(xi)
        x = self.conv14_bn(x)
        x4 = self.relu(x)
        x = torch.cat((x0,x1,x2,x3,x4),1)
        #print(x.shape)
        x = self.avgpool(x)
        #print(x.shape)
        x = x.reshape((-1, 640*15))
        #print(x.shape)
        x = self.fc(x)
        return x

model = InceptionNet()
model = model.float()
model.type(dtype)
model= model.to(device)   
loss_fn = nn.CrossEntropyLoss().type(dtype)
optimizer = optim.Adam(model.parameters(),lr=0.001)

In [11]:
correct = 0
total = 0
for t in range(42):
  for x_batch, y_batch in train_loader:

        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        model.zero_grad()

        y_pred = model(x_batch)

        loss = loss_fn(y_pred, y_batch.type(torch.LongTensor).cuda())
    
        
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(y_pred.data, 1)  # Choose the best class from the output: The class with the best score
        total += y_batch.size(0)                    # Increment the total count
        correct += (predicted == y_batch).sum()     # Increment the correct count
    
  print('Accuracy of the network on the training set: %d %%' % (100 * correct / total))

  #print(loss)
        

Accuracy of the network on the training set: 41 %
Accuracy of the network on the training set: 49 %
Accuracy of the network on the training set: 54 %
Accuracy of the network on the training set: 57 %
Accuracy of the network on the training set: 60 %
Accuracy of the network on the training set: 62 %
Accuracy of the network on the training set: 64 %
Accuracy of the network on the training set: 67 %
Accuracy of the network on the training set: 69 %
Accuracy of the network on the training set: 70 %
Accuracy of the network on the training set: 72 %
Accuracy of the network on the training set: 73 %
Accuracy of the network on the training set: 74 %
Accuracy of the network on the training set: 75 %
Accuracy of the network on the training set: 76 %
Accuracy of the network on the training set: 78 %
Accuracy of the network on the training set: 79 %
Accuracy of the network on the training set: 80 %
Accuracy of the network on the training set: 80 %
Accuracy of the network on the training set: 81 %


In [12]:
correct = 0
total = 0
for x_batch, y_batch in val_loader:

        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        model.zero_grad()

        outputs = model(x_batch)

        _, predicted = torch.max(outputs.data, 1)  # Choose the best class from the output: The class with the best score
        total += y_batch.size(0)                    # Increment the total count
        correct += (predicted == y_batch).sum()     # Increment the correct count
    
print('Accuracy of the network on the test set: %d %%' % (100 * correct / total))

Accuracy of the network on the test set: 69 %


In [0]:
X_test_reshape = np.expand_dims(X_test_cur, axis=1)
x_test_tensor = torch.from_numpy(X_test_reshape).float()
y_test_tensor = torch.from_numpy(y_test).float()

test_data = TensorDataset(x_test_tensor, y_test_tensor)
test_loader = DataLoader(dataset=test_data, batch_size=batchsize, shuffle=True)

In [15]:
correct = 0
total = 0
for x_batch, y_batch in test_loader:

        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        model.zero_grad()
        outputs = model(x_batch)

        _, predicted = torch.max(outputs.data, 1)  # Choose the best class from the output: The class with the best score
        total += y_batch.size(0)                    # Increment the total count
        correct += (predicted == y_batch).sum()     # Increment the correct count
    
print('Accuracy of the network on the test set: %d %%' % (100 * correct / total))

Accuracy of the network on the test set: 70 %
